In [ ]:
from specsscan.core import SpecsScan
import matplotlib.pyplot as plt

In [ ]:
sps = SpecsScan()

In [ ]:
print(sps)

In [ ]:
print(sps.spa)

In [ ]:
from specsscan.settings import parse_config

In [ ]:
parse_config({})

In [ ]:
print(parse_config("../tests/data/config.yaml"))

In [ ]:
config = parse_config("../tests/data/config.yaml")

In [ ]:
import json
with open("../tests/data/config.json", "w") as outfile:
    json.dump(config, outfile)